Depreciated, use source/utils/save_split_model.py instead 

In [2]:
from source.core.engine import MoP
import source.core.run_partition as run_p
from os import environ
from source.utils.dataset import *
from source.utils.misc import *

import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from source.models import resnet

import torch.nn.functional as F

import numpy as np

from source.utils import io
from source.utils import testers
from source.core import engine
import json
import itertools

from torchsummary import summary

import time

c:\Users\natet\anaconda3\envs\cap_nb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path = os.path.join('.','assets', 'models')
model_names = os.listdir(model_path)

for model_name in model_names:
    if 'resnet101' in model_name and '0001' in model_name:
        print(model_name)

cifar100-resnet101-kernel-np4-pr0.5-lcm0.0001.pt
cifar100-resnet101-kernel-np4-pr0.75-lcm0.0001.pt
cifar100-resnet101-kernel-np4-pr0.85-lcm0.0001.pt


In [13]:
'''
    Grab saved model and partition information
'''

# setup config
dataset='cifar10'
environ["config"] = f"config/{dataset}.yaml"

configs = run_p.main()

configs["device"] = "cpu"
configs['load_model'] = "cifar10-resnet18-kernel-np4-pr0.85-lcm1e-05.pt"
configs["num_partition"] = '4' #'resnet18-v2.yaml'

# load data and load or train model
model = get_model_from_code(configs).to(configs['device']) # grabs model architecture from ./source/models/escnet.py
print(model)

# load model params into dictionary
state_dict = torch.load(io.get_model_path("{}".format(configs["load_model"])), map_location=configs['device'])

# load weights into full model
model = io.load_state_dict(model, 
                    state_dict['model_state_dict'] if 'model_state_dict' in state_dict 
                    else state_dict['state_dict'] if 'state_dict' in state_dict else state_dict,)

# gets random test input (with correct size)
input_var = engine.get_input_from_code(configs)
#print(input_var)

# Config partitions and prune_ratio
configs['num_partition'] = '4'#'./config/resnet18-v2.yaml'
configs = engine.partition_generator(configs, model)
            
# Compute output size of each layer
configs['partition'] = engine.featuremap_summary(model, configs['partition'], input_var)
        
# Setup communication costs
configs['comm_costs'] = engine.set_communication_cost(model, configs['partition'],)


device :  
model :  resnet18
data_code :  cifar10
num_classes :  10
model_file :  test.pt
epochs :  0
batch_size :  128
optimizer :  sgd
lr_scheduler :  default
learning_rate :  0.01
seed :  1234
sparsity_type :  kernel
prune_ratio :  1
admm :  True
admm_epochs :  300
rho :  0.0001
multi_rho :  True
retrain_bs :  128
retrain_lr :  0.005
retrain_ep :  50
retrain_opt :  default
xentropy_weight :  1.0
warmup :  False
warmup_lr :  0.001
warmup_epochs :  10
mix_up :  True
alpha :  0.3
smooth :  False
smooth_eps :  0
save_last_model_only :  False
num_partition :  1
layer_type :  regular
bn_type :  masked
par_first_layer :  False
comm_outsize :  False
lambda_comm :  0
lambda_comp :  0
distill_model :  
distill_loss :  kl
distill_temp :  30
distill_alpha :  1
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (c

In [14]:
'''
    Setup saving for split model 
'''

num_machines = int(configs["num_partition"])

print(f'Number of machines = {num_machines}')

# make dir name 
time_stamp = time.strftime("%Y%m%d-%H%M%S")
if len(configs['load_model']) == 0:
    folder_name='vsplit-{}-{}-{}-np{}-pr{}-lcm{}-{}'.format( 
                configs['data-code'], 
                configs['model'], 
                configs['sparsity-type'], 
                configs['num_partition'], 
                configs['prune-ratio'], 
                configs['lambda-comm'],
                time_stamp)
else:
    folder_name = 'vsplit-{}-{}'.format(configs['load_model'][:-3],time_stamp)

# make folder 
folder_path = os.path.join(os.getcwd(), 'assets', 'models',folder_name)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for i in range(num_machines):
    machine_path = os.path.join(folder_path,f'machine-{i}')
    if not os.path.exists(machine_path):
        os.makedirs(machine_path)

print(f'save path = {folder_path}')

Number of machines = 4
save path = c:\Users\natet\Desktop\graduate school\thesis\CaP\assets\models\vsplit-cifar10-resnet18-kernel-np4-pr0.85-lcm1e-05-20240930-235425


In [15]:
'''
    Look at modules that will be split 
'''

module_names =  [module[0] for i, module in enumerate(model.named_modules())]
num_total_modules = len(module_names)

print(f'Total modules {num_total_modules}')

index = 0
for mname in module_names:
    print(f'{index} - {mname}')
    index += 1

split_module_names = list(configs['partition'].keys())
print(f'Number of split modules {len(split_module_names)}')

index = 0
for sname in split_module_names:
    print(f'{index} - {sname}')
    index += 1


Total modules 62
0 - 
1 - conv1
2 - bn1
3 - layer1
4 - layer1.0
5 - layer1.0.conv1
6 - layer1.0.conv2
7 - layer1.0.bn1
8 - layer1.0.bn2
9 - layer1.0.shortcut
10 - layer1.1
11 - layer1.1.conv1
12 - layer1.1.conv2
13 - layer1.1.bn1
14 - layer1.1.bn2
15 - layer1.1.shortcut
16 - layer2
17 - layer2.0
18 - layer2.0.conv1
19 - layer2.0.conv2
20 - layer2.0.bn1
21 - layer2.0.bn2
22 - layer2.0.shortcut
23 - layer2.0.shortcut.0
24 - layer2.0.shortcut.1
25 - layer2.1
26 - layer2.1.conv1
27 - layer2.1.conv2
28 - layer2.1.bn1
29 - layer2.1.bn2
30 - layer2.1.shortcut
31 - layer3
32 - layer3.0
33 - layer3.0.conv1
34 - layer3.0.conv2
35 - layer3.0.bn1
36 - layer3.0.bn2
37 - layer3.0.shortcut
38 - layer3.0.shortcut.0
39 - layer3.0.shortcut.1
40 - layer3.1
41 - layer3.1.conv1
42 - layer3.1.conv2
43 - layer3.1.bn1
44 - layer3.1.bn2
45 - layer3.1.shortcut
46 - layer4
47 - layer4.0
48 - layer4.0.conv1
49 - layer4.0.conv2
50 - layer4.0.bn1
51 - layer4.0.bn2
52 - layer4.0.shortcut
53 - layer4.0.shortcut.0
54 

In [16]:
'''
    Save split layers to diff machines
'''

# iterate through layers 1 module at a time 
for imodule in range(num_total_modules):

        print(f'Saving module {imodule}: {module_names[imodule]}')

        # iterate through each machine (done in parallel later)
        for imach in range(num_machines):
                print(f'\tMachine {imach}')

                # get the current module
                # TODO: this is very bad for latency. Only load module if you have to 
                curr_name, curr_module = next((x for i,x in enumerate(model.named_modules()) if i==imodule)) 

                # update I/O if encounter split layer
                # TODO: revist this implementation
                split_param_name = curr_name + '.weight'
                if split_param_name in split_module_names:

                        # skip if machine doesnt expect input
                        if len(configs['partition'][split_param_name]['channel_id'][imach]) == 0:
                                print(f'\t\t-No input assigned to this machine. Skipping...')
                                continue
                        
                        # TODO: reconsider implementation 
                        # What input channels does this machine compute?
                        input_channels = torch.tensor(configs['partition'][split_param_name]['channel_id'][imach],
                                device=torch.device(configs['device']))
                        N_in = len(input_channels) # TODO: is this used?

                        # Where to send output (map of output channels to different machines)
                        output_channel_map = configs['partition'][split_param_name]['filter_id']

                # reduce computation-- make vertically split layer
                # supports Linear, BN, and Conv layers
                # TODO: generalize this to more than conv layers 
                if type(curr_module) == nn.Conv2d:
                        split_layer = nn.Conv2d(N_in,
                                        curr_module.weight.shape[0], # TODO does this need to be an int? (currently tensor)
                                        kernel_size= curr_module.kernel_size,
                                        stride=curr_module.stride,
                                        padding=curr_module.padding, 
                                        bias=curr_module.bias)

                        # write parameters to split layer 
                        split_layer.weight = torch.nn.Parameter(curr_module.weight.index_select(1, input_channels))


                elif type(curr_module) == nn.BatchNorm2d:
                        split_layer = nn.BatchNorm2d(N_in, 
                                        curr_module.eps,
                                        momentum=curr_module.momentum, 
                                        affine=curr_module.affine, 
                                        track_running_stats=curr_module.track_running_stats)

                        # write parameters to split layer 
                        split_layer.weight = torch.nn.Parameter(curr_module.weight.index_select(0, input_channels))
                        split_layer.running_mean = torch.nn.Parameter(curr_module.running_mean.index_select(0, input_channels))
                        split_layer.running_var = torch.nn.Parameter(curr_module.running_var.index_select(0, input_channels))

                        if not curr_module.bias == None:
                                split_layer.bias = torch.nn.Parameter(curr_module.bias.index_select(0, input_channels))
                        

                elif type(curr_module) == nn.Linear:
                        # TODO: assumes there is a bias 
                        split_layer = nn.Linear(N_in, 
                                        curr_module.weight.shape[0])

                        # write parameters to split layer 
                        split_layer.weight = torch.nn.Parameter(curr_module.weight.index_select(1, input_channels))

                        # TODO: double check bias is applied correctly
                        if not curr_module.bias == None:
                                split_layer.bias = curr_module.bias
                else:
                        print(f'\t\t-Skipping module {type(curr_module).__name__}')
                        continue

                # save split layer
                curr_module_name = module_names[imodule].replace('.', '-')
                curr_machine_path = os.path.join(folder_path,f'machine-{imach}')
                fpath = os.path.join(curr_machine_path, f'{curr_module_name}.pth')
                torch.save(split_layer.state_dict(), fpath)


        print(f'Finished saving module {imodule}')
        print()



Saving module 0: 
	Machine 0
		-Skipping module ResNet
	Machine 1
		-Skipping module ResNet
	Machine 2
		-Skipping module ResNet
	Machine 3
		-Skipping module ResNet
Finished saving module 0

Saving module 1: conv1
	Machine 0
		-No input assigned to this machine. Skipping...
	Machine 1
	Machine 2
	Machine 3
Finished saving module 1

Saving module 2: bn1
	Machine 0
	Machine 1
	Machine 2
	Machine 3
Finished saving module 2

Saving module 3: layer1
	Machine 0
		-Skipping module Sequential
	Machine 1
		-Skipping module Sequential
	Machine 2
		-Skipping module Sequential
	Machine 3
		-Skipping module Sequential
Finished saving module 3

Saving module 4: layer1.0
	Machine 0
		-Skipping module BasicBlock
	Machine 1
		-Skipping module BasicBlock
	Machine 2
		-Skipping module BasicBlock
	Machine 3
		-Skipping module BasicBlock
Finished saving module 4

Saving module 5: layer1.0.conv1
	Machine 0
	Machine 1
	Machine 2
	Machine 3
Finished saving module 5

Saving module 6: layer1.0.conv2
	Machine 0